In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from typing import Union

In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

In [3]:
# Get lines from corridor

def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'SEV', 'CAD']
Sampled route: ['CIU', 'COR', 'SEV', 'CAD']


In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'CIU': (737, 737), 'COR': (790, 794), 'SEV': (876, 883), 'CAD': (921, 921)}

In [26]:
# Generate random requested timetable in corridor for a day t

# Number of services:
n_services = 3

# 1) Generate random timetable
schedule = {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}
schedule

{1: {'PGE': (987, 987), 'ANT': (1067, 1071), 'MAL': (1104, 1104)},
 2: {'MAD': (742, 742),
  'CIU': (785, 788),
  'COR': (871, 873),
  'SEV': (976, 982),
  'CAD': (1037, 1037)},
 3: {'ANT': (208, 208), 'GRA': (241, 241)}}

In [37]:
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [38]:
def get_real_vars(schedule):
    real_vars = []
    for service in schedule:
        for i, stop in enumerate(schedule[service]):
            if i == 0 or i == len(schedule[service]) - 1:
                real_vars.append(schedule[service][stop][0])
            else: 
                real_vars.append(schedule[service][stop][0])
                real_vars.append(schedule[service][stop][1])
                
    return real_vars

real_vars = get_real_vars(schedule)
real_vars

[0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225]

In [39]:
def get_real_boundaries(real_vars):
    real_boundaries = []
    for rv in real_vars:
        real_boundaries.append((rv-10, rv+10))
    
    return real_boundaries
    
real_boundaries = get_real_boundaries(real_vars)
real_boundaries

[(-10, 10),
 (138, 158),
 (142, 162),
 (170, 190),
 (-2, 18),
 (18, 38),
 (20, 40),
 (155, 175),
 (157, 177),
 (200, 220),
 (20, 40),
 (170, 190),
 (172, 192),
 (215, 235)]

In [40]:
def update_schedule(solution):
    """
    Update schedule with the solution
    
    Args:
        solution (list): solution
    """
    s_idx = 0
    for service in schedule:
        for i, stop in enumerate(schedule[service]):
            if i == 0 or i == len(schedule[service]) - 1:
                schedule[service][stop] = (solution[s_idx], solution[s_idx])
                s_idx += 1
            else:
                schedule[service][stop] = (solution[s_idx], solution[s_idx+1])
                s_idx += 2
                
update_schedule(real_vars)
schedule

{1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
 2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
 3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [41]:
def is_feasible(solution, security_gap=10):
    """
    Check if the solution is feasible
    
    Args:
        solution (dict): solution
        security_gap (int, optional): security gap
        
    Returns:
        bool: True if the solution is feasible, False otherwise
    """
    S_i = np.array(solution["discrete"])
    update_schedule(solution["real"])
    security_array = []
    
    # Get conflicts between services
    for service in schedule:
        service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
        for service_k in schedule:
          service_sec_row = [] 
          for stop in schedule[service]:
            if service_k == service or stop not in schedule[service_k]:
              service_sec_row.append(0)
              continue
        
            if abs(schedule[service][stop][1] - schedule[service_k][stop][1]) < security_gap:
              service_sec_row.append(1)
            else:
              service_sec_row.append(0)
          service_sec_arr.append(service_sec_row)
        
        security_array.append(np.array(service_sec_arr))
    
    # Get array of security conflicts. If there is a conflict, the dot product will be different from zero
    if not S_i.dot(np.array([np.sum(S_i.dot(ssa)) for ssa in security_array])):
        return True
    return False

"""
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

solution = {"real": [0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225], "discrete": [1, 1, 1]}

is_feasible(solution)
"""

'\nschedule = {1: {\'MAD\': (0, 0), \'BAR\': (148, 152), \'FIG\': (180, 180)},\n            2: {\'MAD\': (8, 8), \'ZAR\': (28, 30), \'BAR\': (165, 167), \'FIG\': (210, 210)},\n            3: {\'MAD\': (30, 30), \'BAR\': (180, 182), \'FIG\': (225, 225)}}\n\nsolution = {"real": [0, 148, 152, 180, 8, 28, 30, 165, 167, 210, 30, 180, 182, 225], "discrete": [1, 1, 1]}\n\nis_feasible(solution)\n'

In [42]:
def get_num_trains(solution):
    scheduled_trains = np.sum(solution["discrete"])
    print("Requested number of trains: ", len(solution["discrete"]))
    print("Scheduled trains: ", scheduled_trains)
    return scheduled_trains, 0

In [90]:
boundaries = {'real': real_boundaries, 'discrete': [(0, 1) for _ in range(len(schedule))]}

gsa_algo = GSA(objective_function=get_num_trains,
               is_feasible=is_feasible,
               r_dim=len(real_vars),
               d_dim=len(schedule),
               boundaries=boundaries)

In [91]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=10,
                                     chaotic_constant=False,
                                     repair_solution=True)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
GSA is optimizing  "get_num_trains"
Requested number of trains:  3
Scheduled trains:  0
Requested number of trains:  3
Scheduled trains:  1
Requested number of trains:  3
Scheduled trains:  1
Requested number of trains:  3
Scheduled trains:  2
Requested number of trains:  3
Scheduled trains:  1
####################################################################################################
REPAIRING SOLUTION... 
####################################################################################################
####################################################################################################
SOLUTION SUCCESSFULLY REPAIRED!!
####################################################################################################
####################################################################################################
REPAIR

In [92]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,2,0,0.000606,"[1, 1, 1]","[3.7708545175161983, 144.4673422532487, 131.92..."
1,1,2,0,0.002348,"[1, 1, 1]","[3.7708545175161983, 144.4673422532487, 131.92..."
2,2,2,0,0.004394,"[1, 1, 1]","[3.7708545175161983, 144.4673422532487, 131.92..."
3,3,2,0,0.005943,"[1, 1, 1]","[3.7708545175161983, 144.4673422532487, 131.92..."
4,4,3,0,0.007604,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."
5,5,3,0,0.009200,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."
6,6,3,0,0.055025,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."
7,7,3,0,0.056787,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."
8,8,3,0,0.057594,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."
9,9,3,0,0.058297,"[1, 1, 1]","[1.650570971808942, 157.8023504800983, 131.786..."


In [93]:
discrete_solution = list(training_history["Discrete"].iloc[-1])
discrete_solution

[1, 1, 1]

In [94]:
schedule_solution = list(training_history["Real"].iloc[-1])
schedule_solution

[1.650570971808942,
 157.8023504800983,
 131.78634090757532,
 177.33626125416401,
 31.049785357994253,
 49.34081116524011,
 24.485325277924076,
 186.4733349428041,
 173.51058574419844,
 216.4480526170515,
 82.06955790841465,
 157.49024871021487,
 204.71696584284655,
 229.47864438131282]

In [95]:
update_schedule(schedule_solution)
schedule

{1: {'MAD': (1.650570971808942, 1.650570971808942),
  'BAR': (157.8023504800983, 131.78634090757532),
  'FIG': (177.33626125416401, 177.33626125416401)},
 2: {'MAD': (31.049785357994253, 31.049785357994253),
  'ZAR': (49.34081116524011, 24.485325277924076),
  'BAR': (186.4733349428041, 173.51058574419844),
  'FIG': (216.4480526170515, 216.4480526170515)},
 3: {'MAD': (82.06955790841465, 82.06955790841465),
  'BAR': (157.49024871021487, 204.71696584284655),
  'FIG': (229.47864438131282, 229.47864438131282)}}

In [96]:
is_feasible({"real": schedule_solution, "discrete": discrete_solution})

True